# Object Classifier

## Process images

### Extract data from .7z files

In [ ]:
from zipfile import ZipFile
dataset = ZipFile('data/cifar-10.zip')
dataset.extractall(path='data')
dataset.close()
print("Dataset extracted successfully")

Dataset extracted successfully


In [ ]:
import py7zr

archive = py7zr.SevenZipFile('data/train.7z', mode='r')
archive.extractall(path='data')
archive.close()
train_file_location='data/train'

archive = py7zr.SevenZipFile('data/test.7z', mode='r')
archive.extractall(path='data')
archive.close()
test_file_location='data/test'


In [ ]:
#imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [ ]:
filenames = os.listdir("data/train")
print(len(filenames))

50000


### Process Data

In [ ]:
def get_image_as_array(path):
    image = Image.open(path)
    return np.array(image)

df = pd.read_csv('data/trainLabels.csv')
df['image'] = df['id'].apply(lambda x: get_image_as_array(f'data/train/{x}.png'))
df.head()

# df['image'] = df['id'].apply(get_image_as_array)
# df.head()





,id,label,image
0,1,frog,"[[[59, 62, 63], [43, 46, 45], [50, 48, 43], [6..."
1,2,truck,"[[[154, 177, 187], [126, 137, 136], [105, 104,..."
2,3,truck,"[[[255, 255, 255], [253, 253, 253], [253, 253,..."
3,4,deer,"[[[28, 25, 10], [37, 34, 19], [38, 35, 20], [4..."
4,5,automobile,"[[[170, 180, 198], [168, 178, 196], [177, 185,..."


In [ ]:
object_classes = df['label'].unique()
print(object_classes)

['frog' 'truck' 'deer' 'automobile' 'bird' 'horse' 'ship' 'cat' 'dog'
 'airplane']


In [ ]:


entries_per_class = df['label'].value_counts()
print(entries_per_class) # no need to standardize or scale

label
frog          5000
truck         5000
deer          5000
automobile    5000
bird          5000
horse         5000
ship          5000
cat           5000
dog           5000
airplane      5000
Name: count, dtype: int64


In [ ]:
# labels_dictionary = dict(enumerate(object_classes))
# print(labels_dictionary)
labels_map = {}
for index, item in enumerate(object_classes):
    labels_map[item] = index

print(labels_map)

{'frog': 0, 'truck': 1, 'deer': 2, 'automobile': 3, 'bird': 4, 'horse': 5, 'ship': 6, 'cat': 7, 'dog': 8, 'airplane': 9}


In [ ]:
df['label'] = df['label'].map(labels_map)
df['image'] = df['image']/255

df.head()

,id,label,image
0,1,0,"[[[0.23137254901960785, 0.24313725490196078, 0..."
1,2,1,"[[[0.6039215686274509, 0.6941176470588235, 0.7..."
2,3,1,"[[[1.0, 1.0, 1.0], [0.9921568627450981, 0.9921..."
3,4,2,"[[[0.10980392156862745, 0.09803921568627451, 0..."
4,5,3,"[[[0.6666666666666666, 0.7058823529411765, 0.7..."


In [ ]:
num_rows_80_percent = int(len(df) * 0.8)

train = df.iloc[:num_rows_80_percent]
test = df.iloc[num_rows_80_percent:]

print(train.shape)
print(test.shape)


(40000, 3)
(10000, 3)


In [53]:
X_train = list(train['image'])
X_test = list(test['image'])

Y_train = list(train['label'])
Y_test = list(test['label'])

print(X_train[0])
print(Y_train[0])


[[[0.23137255 0.24313725 0.24705882]
  [0.16862745 0.18039216 0.17647059]
  [0.19607843 0.18823529 0.16862745]
  ...
  [0.61960784 0.51764706 0.42352941]
  [0.59607843 0.49019608 0.4       ]
  [0.58039216 0.48627451 0.40392157]]

 [[0.0627451  0.07843137 0.07843137]
  [0.         0.         0.        ]
  [0.07058824 0.03137255 0.        ]
  ...
  [0.48235294 0.34509804 0.21568627]
  [0.46666667 0.3254902  0.19607843]
  [0.47843137 0.34117647 0.22352941]]

 [[0.09803922 0.09411765 0.08235294]
  [0.0627451  0.02745098 0.        ]
  [0.19215686 0.10588235 0.03137255]
  ...
  [0.4627451  0.32941176 0.19607843]
  [0.47058824 0.32941176 0.19607843]
  [0.42745098 0.28627451 0.16470588]]

 ...

 [[0.81568627 0.66666667 0.37647059]
  [0.78823529 0.6        0.13333333]
  [0.77647059 0.63137255 0.10196078]
  ...
  [0.62745098 0.52156863 0.2745098 ]
  [0.21960784 0.12156863 0.02745098]
  [0.20784314 0.13333333 0.07843137]]

 [[0.70588235 0.54509804 0.37647059]
  [0.67843137 0.48235294 0.16470588]


## Create Neural Net

We shall create the Neural Network

In [54]:
import tensorflow as tf
from tensorflow import keras



In [56]:
num_of_classes = len(object_classes)

#setting up layers

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32,32,3)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(num_of_classes, activation='softmax')
])

/Users/ansinha/Library/Python/3.9/lib/python/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
